In [9]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers
import tensorflow.keras.utils as ku 
import numpy as np
import matplotlib.pyplot as plt

In [10]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
tokenizer = Tokenizer()
data = open('/content/drive/MyDrive/Final Exam - Tipe B/Question 4/poem.txt').read()

corpus = data.lower().split("\n")


tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) + 1

# create input sequences using list of tokens
input_sequences = []
for line in corpus:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)


# pad sequences 
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

# create predictors and label
predictors, label = input_sequences[:,:-1],input_sequences[:,-1]

label = ku.to_categorical(label, num_classes=total_words)

print(max_sequence_len)

16


In [12]:
#buatlah model dengan output layer total words/2 dan total word
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
model.add(Bidirectional(LSTM(150, return_sequences = True)))
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dense(total_words/2, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(Dense(total_words, activation='softmax'))

In [13]:
#Gunakan loss categorical_crossentropy
model.compile(
    loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy']
)

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 15, 100)           380800    
                                                                 
 bidirectional (Bidirectiona  (None, 15, 300)          301200    
 l)                                                              
                                                                 
 dropout (Dropout)           (None, 15, 300)           0         
                                                                 
 lstm_1 (LSTM)               (None, 100)               160400    
                                                                 
 dense (Dense)               (None, 1904)              192304    
                                                                 
 dense_1 (Dense)             (None, 3808)              7254240   
                                                        

In [18]:
#training model 
# Biar cantik aja progress bar nya
import sys
!{sys.executable} -m pip install tensorflow-addons
import tensorflow_addons as tfa
tqdm_callback = tfa.callbacks.TQDMProgressBar()

history = model.fit(
    predictors, label, epochs=20, verbose=0, callbacks=[tqdm_callback]
)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Training:   0%|           0/20 ETA: ?s,  ?epochs/s

Epoch 1/20


0/510           ETA: ?s - 

Epoch 2/20


...

0/510           ETA: ?s - 

Epoch 19/20


0/510           ETA: ?s - 

Epoch 20/20


0/510           ETA: ?s - 

In [19]:
#save model
model.save('model4.h5')